In [1]:
import math as math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import mapping

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)


import warnings
warnings.filterwarnings('ignore')

In [3]:
import route_dynamics.route_elevation.base_df as base
import route_dynamics.route_energy.longi_dynam_model as ldm
from route_dynamics.route_riders import route_riders as ride
from route_dynamics.route_visualizer import visualizer as vis
import route_dynamics.route_energy.knn as knn

In [4]:
shapefile_name = '../data/rt22_pts2.shp'
signals_shp = '../data/traffic_signals2.shp'

In [5]:
data = pd.read_csv("../data/acceleration.csv", names=['time (s)', 'accel. (g)'])

In [6]:
data['time (s)'] = data['time (s)'].round(0)
data['accel. (g)'] = data['accel. (g)'].round(4)

In [7]:
#Convert to SI units
data['accel. (m/s^2)'] = data['accel. (g)']*9.81

In [8]:
#Calculate Velocity

data['vel. (m/s)'] = np.zeros(len(data.index))

for i in range(1, len(data)):
    data['vel. (m/s)'][i] = ((data['accel. (m/s^2)'][i] + data['accel. (m/s^2)'][i-1])/2*1) + data['vel. (m/s)'][i-1]
    
data['vel. (mph)'] = data['vel. (m/s)']*2.2

In [9]:
#Calculate Distance

data['dist. (m)'] = np.zeros(len(data.index))

for i in range(1, len(data)):
    data['dist. (m)'][i] = ((data['vel. (m/s)'][i] + data['vel. (m/s)'][i-1])/2*1) + data['dist. (m)'][i-1]

In [16]:
data['dist (ft)'] = data['dist. (m)']*3.281
data.head(10)

,time (s),accel. (g),accel. (m/s^2),vel. (m/s),vel. (mph),dist. (m),dist (ft)
0,0.0,0.0006,0.005886,0.000000,0.000000,0.000000,0.000000
1,1.0,0.1156,1.134036,0.569961,1.253914,0.284981,0.935021
2,2.0,0.1239,1.215459,1.744709,3.838359,1.442315,4.732236
3,3.0,0.1261,1.237041,2.970959,6.536109,3.800149,12.468288
4,4.0,0.1256,1.232136,4.205547,9.252203,7.388402,24.241345
5,5.0,0.1217,1.193877,5.418553,11.920818,12.200452,40.029682
6,6.0,0.1144,1.122264,6.576624,14.468573,18.198041,59.707771
7,7.0,0.1078,1.057518,7.666515,16.866333,25.319610,83.073640
8,8.0,0.0994,0.975114,8.682831,19.102228,33.494283,109.894743
9,9.0,0.0922,0.904482,9.622629,21.169784,42.647013,139.924850


In [25]:
#Create new dataframe where dist. is the indep var

data2 = pd.DataFrame({'dist. (ft)': np.arange(0, 1260, 36)})
data2['dist. (m)'] = data2['dist. (ft)']/3.281

data2['vel. (m/s)'] = np.zeros(len(data2.index))

data2['vel. (km/hr)'] = np.zeros(len(data2.index))

data2['vel. (mph)'] = np.zeros(len(data2.index))

data2['accel. (m/s^2)'] = np.zeros(len(data2.index))

data2['time (s)'] = np.zeros(len(data2.index))

In [26]:
data2.head()

,dist. (ft),dist. (m),vel. (m/s),vel. (km/hr),vel. (mph),accel. (m/s^2),time (s)
0,0,0.000000,0.0,0.0,0.0,0.0,0.0
1,36,10.972265,0.0,0.0,0.0,0.0,0.0
2,72,21.944529,0.0,0.0,0.0,0.0,0.0
3,108,32.916794,0.0,0.0,0.0,0.0,0.0
4,144,43.889058,0.0,0.0,0.0,0.0,0.0


In [17]:
z = data2['dist. (ft)'].iloc[2]
params = data.iloc[(data['dist (ft)']-z).abs().argsort()[:1]]
row = params.index.values

#linear interpolation
vel = ((data['vel. (m/s)'][row].values - data['vel. (m/s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['vel. (m/s)'][row-1].values


accel = ((data['accel. (m/s^2)'][row].values - data['accel. (m/s^2)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['accel. (m/s^2)'][row-1].values


In [21]:
vel

array([7.15101368])

In [28]:
for i in range(1, len(data2)):

    z = data2['dist. (ft)'].iloc[i]
    params = data.iloc[(data['dist (ft)']-z).abs().argsort()[:1]]
    row = params.index.values


    #Interpolate
    if params['dist (ft)'].values < z:
        vel = ((data['vel. (m/s)'][row+1].values - data['vel. (m/s)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['vel. (m/s)'][row].values
        accel = ((data['accel. (m/s^2)'][row+1].values - data['accel. (m/s^2)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['accel. (m/s^2)'][row].values
        time = ((data['time (s)'][row+1].values - data['time (s)'][row].values)/(data['dist. (m)'][row +1].values - data['dist. (m)'][row].values) * (z/3.28 - data['dist. (m)'][row].values)) + data['time (s)'][row].values
    else:
        vel = ((data['vel. (m/s)'][row].values - data['vel. (m/s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['vel. (m/s)'][row-1].values
        accel = ((data['accel. (m/s^2)'][row].values - data['accel. (m/s^2)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['accel. (m/s^2)'][row-1].values
        time = ((data['time (s)'][row].values - data['time (s)'][row-1].values)/(data['dist. (m)'][row].values - data['dist. (m)'][row-1].values) * (z/3.28 - data['dist. (m)'][row-1].values)) + data['time (s)'][row-1].values
        
    
    data2['vel. (m/s)'].iloc[i] = vel
    data2['vel. (km/hr)'].iloc[i] = vel * 3.6
    data2['vel. (mph)'].iloc[i] = vel*2.2
    data2['accel. (m/s^2)'].iloc[i] = accel
    data2['time (s)'].iloc[i] = time

In [29]:
data2.head()

,dist. (ft),dist. (m),vel. (m/s),vel. (km/hr),vel. (mph),accel. (m/s^2),time (s)
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,36,10.972265,5.109799,18.395277,11.241558,1.203615,4.745464
2,72,21.944529,7.151014,25.743649,15.732230,1.088142,6.527016
3,108,32.916794,8.612282,31.004216,18.947021,0.980834,7.930584
4,144,43.889058,9.731477,35.033317,21.409250,0.896401,9.124810


In [30]:
len(data2)

35

In [ ]:
route_df = base.wrapper(shapefile_name, 6, 6)
route_df.head()

In [ ]:
# Define cutoff distance for acceleration and deceleration
a_pos = 0.4
a_neg = -1.5
a_avg = data2['accel. (m/s^2)'].mean()
a_plt = data2['accel. (m/s^2)'].iloc[len(data2)-1]
x_a = route_df['speed_limit'].iloc[i]**2. / (2*a_avg)
x_d = route_df['speed_limit'].iloc[i]**2. / (2*a_neg)

In [ ]:
x_p = data2['dist. (m)'].iloc[34]

x_ns = np.zeros(len(route_df.index)) #next stops
x_ls = np.zeros(len(route_df.index)) # prev. stops

In [ ]:
trip_df = ride.route_ridership('PM', 22, 14000)
geometry = trip_df.geometry.values

In [ ]:
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i][0])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df

stop_coords = xy_df.coordinates.values

In [ ]:
signals = gpd.read_file(signals_shp)
signals = signals[signals['Route_Num']==str(22)]
geometry = signals.geometry.values
xy = []
for i in range(len(geometry)):
    dic = mapping(geometry[i])
    coords = dic['coordinates']
    xy.append(coords)
    xy_df = pd.DataFrame(columns = ['STOP_ID','coordinates'])
    xy_df['coordinates'] = xy
    
xy_df

signal_coords = xy_df.coordinates.values

In [ ]:
stop_nn_indicies, stop_coord_nn = knn.find_knn(
1,
route_df.geometry.values,
stop_coords
)


signal_nn_indicies, singal_coord_nn = knn.find_knn(1,route_df.geometry.values, signal_coords)

route_df = route_df.assign(
is_bus_stop = ([False] * len(route_df.index))
)

route_df = route_df.assign(
is_signal = ([False] * len(route_df.index))
)

route_df = route_df.assign(
is_stop = ([False] * len(route_df.index))
)

for i in stop_nn_indicies.ravel():
    route_df.at[i, 'is_bus_stop'] = True
    route_df.at[i, 'is_stop'] = True
    
for i in signal_nn_indicies.ravel()[::2]:
    route_df.at[i, 'is_stop'] = True
    route_df.at[i, 'is_signal'] = True

In [ ]:
route_df.head()

In [ ]:
for i in range(len(x_ns)):
    # set values to Nan if bus stop
    if route_df.at[i, 'is_stop']:
        x_ns[i] = 0.
        x_ls[i] = 0.
        # move to next point
        continue
    else:
        # Calculate 'x_ns';
        # Iterate through remaining indicies to count distance to
        # next stop.
        for j in range(i+1, len(x_ns)):
            # add distance to next point to 'x_ns'
            x_ns[i] += 10.973 
            if route_df.at[j, 'is_stop']:
                break # done calulating 'x_ns' at this point
            # elif not bus stop: move to next point, add distance

        # Calculate 'x_ls';
        # Iterate through previous indicies to cout distance to
        # last stop.
        for j in range(i, 0, -1):
            # Inclusive start to range because distances are
            # backward difference. Dont need to include 'j=0'
            # because the first point has no backward difference.
            if route_df.at[j, 'is_stop']:
                break # done calulating x_ls at this point
            x_ls[i] += 10.973

In [ ]:
x_ls

In [ ]:
x_ns

In [ ]:
v = np.zeros(len(route_df.index)) #array for vel.
a = np.zeros(len(route_df.index)) #array for accel.

count = 0

In [ ]:
v_lim = route_df['speed_limit'].iloc[i]

for i in range(len(x_ns)):

    if count > i:
        continue

    else:

        #Case 1

        if (
            x_ns[i]<=abs(x_d)
            and
            not route_df.at[i, 'is_stop']
            ):

            if x_ns[i] < x_ls[i]:

                a[i] = a_neg
                v[i] = np.sqrt(-2*x_ns[i]*a_neg)
                count += 1

            else:
                for j in range(len(data2)):

                    if count < len(x_ns):

                        if v[count-1] < v_lim:
                            a[count] = data2['accel. (m/s^2)'].iloc[j]
                            v[count] = data2['vel. (m/s)'].iloc[j]
                            count+=1

                        elif v[count-1]>v_lim:

                            while x_ns[count]>abs(x_d):

                                a[count] = 0
                                v[count] = v_lim

                                count+=1

                            else:
                                continue

                        else:
                            continue

                    else:
                        break
                        
            #Case 2

        elif (
            x_ns[i] > abs(x_d)
            and
            x_ls[i] >= x_p
            and
            not route_df.at[i, 'is_stop']
            ):

            if v[i-1] < v_lim:

                a[i] = a_pos
                v[i] = np.sqrt(2*x_ns[i]*a[i])
                count += 1

            elif v[i-1]>v_lim:

                while x_ns[count]>abs(x_d):

                    a[count] = 0
                    v[count] = v_lim

                    count+=1

                else:
                    continue 

            else:
                continue
        #Case 3

        elif (
            x_ls[i] < x_p
            and
            x_ns[i] > abs(x_d)
            and
            not route_df.at[i, 'is_stop']
            ):

            for j in range(len(data2)):

                if count < len(x_ns):

                    if v[count-1] < v_lim:
                        a[count] = data2['accel. (m/s^2)'].iloc[j]
                        v[count] = data2['vel. (m/s)'].iloc[j]
                        count+=1

                    elif v[count-1]>v_lim:

                        while x_ns[count]>abs(x_d):

                            a[count] = 0
                            v[count] = v_lim

                            count+=1

                        else:
                            continue

                    else:
                        continue

                else:
                    break
                    
        elif route_df.at[i, 'is_stop']:
            count += 1 

In [ ]:
back_diff_delta_x = np.full(len(route_df), 10.973)


segment_avg_velocities = (
                v
                +
                np.append(0,v[:-1])
                )/2

delta_times = back_diff_delta_x / segment_avg_velocities
delta_times[delta_times > 1000000] = 0

for i in range(len(route_df)):
    if route_df.at[i, 'is_stop']:
        delta_times[i]+=30 #make variable later
    else:
        pass

time_on_route = np.append(0, np.cumsum(delta_times[1:]))

t = time_on_route
total_time = t[-1]
total_time/60
#should be 62 min

In [ ]:
plt.figure(figsize=(18, 8))

plt.plot(t/60, a)
plt.ylabel('Accel. [m/s^2]')
plt.xlabel('Time [min]')
plt.show()

In [ ]:
plt.figure(figsize=(18, 8))

plt.plot(t/60, v*2.24)
plt.ylabel('Vel. [mph]')
plt.xlabel('Time [min]')
plt.show()

In [ ]:
np.sum(power*delta_t)

In [ ]:
energy = np.sum(power * delta_t)

In [ ]:
energy

In [ ]:
ex22.energy_from_route() 

In [ ]:
v_lim = 15 #m/s or ~30 mph
a_neg = -0.4

# Define cutoff distance for acceleration and deceleration
a_avg = data2['accel. (m/s^2)'].mean()
a_plt = data2['accel. (m/s^2)'].iloc[len(data2)-1]
x_a = v_lim**2. / (2*a_avg)

# Define cutoff distance for acceleration and deceleration
x_d = v_lim**2. / (2*a_neg)

# Distance of accel profile

x_p = len(data2)*0.3048


In [ ]:
x_ns = np.zeros(len(route_df.index)) #next stops
x_ls = np.zeros(len(route_df.index)) # prev. stops

for i in range(len(x_ns)):
    # set values to Nan if bus stop
    if route_df.at[i, 'is_bus_stop']:
        x_ns[i] = 0.
        x_ls[i] = 0.
        # move to next point
        continue
    else:
        # Calculate 'x_ns';
        # Iterate through remaining indicies to count distance to
        # next stop.
        for j in range(i+1, len(x_ns)):
            # add distance to next point to 'x_ns'
            x_ns[i] += 10.973
            if route_df.at[j, 'is_bus_stop']:
                break # done calulating 'x_ns' at this point
            # elif not bus stop: move to next point, add distance

        # Calculate 'x_ls';
        # Iterate through previous indicies to cout distance to
        # last stop.
        for j in range(i, 0, -1):
            # Inclusive start to range because distances are
            # backward difference. Dont need to include 'j=0'
            # because the first point has no backward difference.
            if route_df.at[j, 'is_bus_stop']:
                break # done calulating x_ls at this point
            x_ls[i] += 10.973


In [ ]:
v = np.zeros(len(route_df.index)) #array for vel.
a = np.zeros(len(route_df.index)) #array for accel.

count = 0

for i in range(len(x_ns)):

    if count > i:
        continue

    else:

        #Case 1

        if (
            x_ns[i]<=abs(x_d)
            and
            not route_df.at[i, 'is_bus_stop']
            ):
            
            if x_ns[i] < x_ls[i]:

                a[i] = a_neg
                v[i] = np.sqrt(-2*x_ns[i]*a_neg)
                count += 1
                
            else:
                for j in range(len(data2)):

                    if count < len(x_ns):

                        if v[count-1] < v_lim:
                            a[count] = data2['accel. (m/s^2)'].iloc[j]
                            v[count] = data2['vel. (m/s)'].iloc[j]
                            count+=1

                        elif v[count-1]>v_lim:

                            while x_ns[count]>abs(x_d):

                                a[count] = 0
                                v[count] = v_lim

                                count+=1

                            else:
                                continue

                        else:
                            continue

                    else:
                        break


        #Case 2

        elif (
            x_ns[i] > abs(x_d)
            and
            x_ls[i] >= x_p
            and
            not route_df.at[i, 'is_bus_stop']
            ):

            if v[i-1] < v_lim:

                a[i] = a_plt
                v[i] = np.sqrt(2*x_ns[i]*a[i])
                count += 1

            elif v[i-1]>v_lim:
                        
                while x_ns[count]>abs(x_d):

                    a[count] = 0
                    v[count] = v_lim

                    count+=1

                else:
                    continue 

            else:
                continue

                    

        #Case 3

        elif (
            x_ls[i] < x_p
            and
            x_ns[i] > abs(x_d)
            and
            not route_df.at[i, 'is_bus_stop']
            ):

            for j in range(len(data2)):

                if count < len(x_ns):

                    if v[count-1] < v_lim:
                        a[count] = data2['accel. (m/s^2)'].iloc[j]
                        v[count] = data2['vel. (m/s)'].iloc[j]
                        count+=1

                    elif v[count-1]>v_lim:
                        
                        while x_ns[count]>abs(x_d):
                
                            a[count] = 0
                            v[count] = v_lim

                            count+=1
                            
                        else:
                            continue
                            
                    else:
                        continue

                else:
                    break


        elif route_df.at[i, 'is_bus_stop']:

            count += 1

In [ ]:
ex22.route_df

In [ ]:
d = {'dist': ex22.route_df.cum_distance, 'last': x_ls, 'next': x_ns, 'stop': ex22.route_df.is_bus_stop, 'accel': a, 'vel':v}

In [ ]:
test = pd.DataFrame(d)

In [ ]:
print (x_d, x_p)

In [ ]:
test.iloc[330:370]

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (8,6), dpi=100)

ax1.plot(ex22.route_df.cum_distance, v, label = 'vel')
ax2.plot(ex22.route_df.cum_distance, a, label = 'accel')
#ax.set_xlabel('time (s)') 
#ax.set_ylabel('Power (kW)')
#ax.legend()